In [1]:
from google.cloud import bigquery

# Initialize BigQuery Client
client = bigquery.Client.from_service_account_json("secrets/serviceKey.json")

project_id = "idmpproject-441123"
dataset_id = "uberFareEstimation"
table_id = 'uber_data'

table_path = f"{project_id}.{dataset_id}.{table_id}"

# Define the query
query = f"""
    SELECT *
    FROM `{table_path}`
    LIMIT 1000
"""

# Run the query
query_job = client.query(query)

# Wait for the query to complete and fetch results
results = query_job.result()

df = results.to_dataframe()
print(df.head())


/Users/sabarimathavan/Downloads/Python/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


   distance cab_type          time_stamp         destination         source  \
0      0.94     Uber 2018-11-28 23:30:00           North End  North Station   
1      0.94     Uber 2018-12-14 19:30:00           North End  North Station   
2      0.63     Uber 2018-11-27 21:30:00  Financial District  South Station   
3      0.63     Uber 2018-12-15 15:00:00  Financial District  South Station   
4      0.63     Uber 2018-12-15 13:30:00  Financial District  South Station   

   price  surge_multiplier      id      name  
0    4.5               1.0   39765  UberPool  
1    4.5               1.0  437984  UberPool  
2    4.5               1.0    1644  UberPool  
3    4.5               1.0   10780  UberPool  
4    4.5               1.0   21598  UberPool  
